# [Python] Crypto: REST API & candlestick graph

Hello :) !

In this exercise, we use the <a href="https://www.coingecko.com/en/api?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2022-01-01">CoinGecko API</a> to create interactive candlestick graphs for Bitcoin with Plotly.

I will use the API to get the price data for 90 days with 24 observation per day, 1 per hour. We will find the max, min, open, and close price per day meaning we will have 90 candlesticks and use that to generate the candlestick graph. We will add a 7-day moving average.

The Python client/wrapper for the API called <a href="https://github.com/man-c/pycoingecko?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2022-01-01">PyCoinGecko</a>  will make the requests easing to make.

## Table of contents:
* [Settings](#zero)
* [Data collection](#one)
* [Wrangling](#two)
* [Visualisation](#three)


<h2> 0. Settings </h2>  <a class="anchor" id="zero"></a>

In [ ]:
!pip install pycoingecko
!pip install plotly
!pip install mplfinance

In [21]:
import pandas as pd
#import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot
import matplotlib.pyplot as plt
import datetime
from pycoingecko import CoinGeckoAPI
from mplfinance.original_flavor import candlestick2_ohlc

<h2> 1. Data collection </h2> <a class="anchor" id="one"></a>

Let's collect the data we need !

Using the <code>get_coin_market_chart_by_id(id, vs_currency, days)</code>. <code>id</code> is the name of the coin you want, <code>vs_currency</code> is the currency you want the price in, and <code>days</code> is how many days back from today you want.

In [72]:
cg = CoinGeckoAPI()

bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=90)

bitcoin_data is a JSON, and includes the price, market caps, and total volumes along with timestamps. 

We focus on the prices:

In [73]:
bitcoin_data_prices = bitcoin_data['prices']
print(bitcoin_data_prices[0:10])

[[1656072050776, 21166.18246916643], [1656075667232, 21197.282092934343], [1656079316105, 21287.049699605162], [1656082891427, 21107.39240252371], [1656086448394, 20890.068373677685], [1656090143060, 21029.17051256745], [1656093700728, 20994.23186749934], [1656097323787, 21136.46103986765], [1656100851292, 21241.45068568767], [1656104514766, 21209.320784739168]]


<h2> 2. Data wrangling </h2> <a class="anchor" id="two"></a>

We convert the data into a pandas dataframe, and convert the TimeStamp into a readable datetime.

In [74]:
data = pd.DataFrame(bitcoin_data_prices, columns=['TimeStamp', 'Price'])
data.head()

,TimeStamp,Price
0,1656072050776,21166.182469
1,1656075667232,21197.282093
2,1656079316105,21287.049700
3,1656082891427,21107.392403
4,1656086448394,20890.068374


In [77]:
data['date'] = data['TimeStamp'].apply(lambda d: datetime.date.fromtimestamp(d/1000))
print(data.date)

0       2022-06-24
1       2022-06-24
2       2022-06-24
3       2022-06-24
4       2022-06-24
           ...    
2156    2022-09-22
2157    2022-09-22
2158    2022-09-22
2159    2022-09-22
2160    2022-09-22
Name: date, Length: 2161, dtype: object


In [78]:
candlestick_data = data.groupby(data.date, as_index=False).agg({"Price": ['min', 'max', 'first', 'last']})
print(candlestick_data.head(5))

         date         Price                                          
                        min           max         first          last
0  2022-06-24  20890.068374  21287.049700  21166.182469  21209.320785
1  2022-06-25  21052.234957  21501.718372  21325.243230  21233.424676
2  2022-06-26  21236.769754  21637.931949  21498.410334  21378.457584
3  2022-06-27  20741.872124  21475.850907  21389.480571  20926.490849
4  2022-06-28  20275.064146  21108.771855  20831.206133  20297.896846


<h2> 3. Data visualisation </h2> <a class="anchor" id="three"></a>

Let's make a first candlestick graph:

In [79]:
fig = go.Figure(data=[go.Candlestick(x=candlestick_data['date'],
                open=candlestick_data['Price']['first'], 
                high=candlestick_data['Price']['max'],
                low=candlestick_data['Price']['min'], 
                close=candlestick_data['Price']['last'])])

fig.show()

We will improve it a bit: get rid of the range slider, add a title, an axis title, add a 7-day moving average....

In [80]:
candlestick_data['7dma'] = candlestick_data['Price']['last'].rolling(window = 7,center = True).mean()

fig.update_layout(xaxis_rangeslider_visible=False, 
template = "simple_white", 
title = "Evolution of bitcoin price (in USD) in the last 90 days", 
yaxis_title = "USD",
legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="center",
    x=0.5
))

fig.data[0].name="Candlesticks"

fig.add_trace(go.Scatter(x = candlestick_data['date'], 
    y = candlestick_data['7dma'],
    line = dict(color = "#000000"),
    name = "7-day moving average"))
fig.show()

Et voilà :) ! 

We learn to retrieve data using CoinGecko API, to prepare and aggregate the data using pandas, and to visualise it using plotly. 